In [86]:
# Julia 0.7
using CSV, DataFrames, Gadfly, Statistics, LinearAlgebra, GLM, CategoricalArrays,StatsBase

# Chargement du jeu de données d'entraînement

In [87]:
train = CSV.read("train.csv")

head(train)

,ID,Name,Platform,Year_of_Release,Genre,Publisher,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,NA_Sales,Global_Sales
,Int64⍰,String⍰,String⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰
1,1,Zoo Keeper,DS,2004,Puzzle,Ignition Entertainment,0.0505242,0.0100241,74,40,7.5,13,Buddiez- Inc.,E,0.101432,0.172605
2,2,Do-Konjou Shougakussei: Bon Bita - Hadaka no Choujou Ketsusen!! Bita vs. Dokuro Dei!,DS,2010,Action,Namco Bandai Games,0.0392108,0.00179053,missing,missing,missing,missing,missing,missing,0.000439844,0.0432509
3,3,Power Play Pool,DS,2006,Sports,System 3 Arcade Software,0.000374026,0.00179738,missing,missing,missing,missing,System 3,E,0.0308291,0.0412706
4,4,SpongeBob SquarePants: Revenge of the Flying Dutchman,PS2,2002,Platform,THQ,0.00138205,0.14001,missing,missing,6.6,41,Big Sky Software,E,0.52833,1.08082
5,5,Deception IV: Blood Ties,PSV,2014,Action,Tecmo Koei,0.0694222,0.00953506,67,19,7.5,61,Koei Tecmo Games,M,0.0206288,0.131074
6,6,Binary Domain,PS3,2012,Action,Sega,0.14039,0.0302705,72,33,8.2,179,Sega,M,0.0890602,0.331157


In [88]:
NA_Sales= collect(train[:,:NA_Sales])
Global_Sales = collect(train[:,:Global_Sales])
Critic_Score = collect(train[:,:Critic_Score])
Critic_Count = collect(train[:,:Critic_Count])
User_Score = collect(train[:,:User_Score])
User_Count = collect(train[:,:User_Count])
Other_Sales = collect(train[:,:Other_Sales])
JP_Sales = collect(train[:,:JP_Sales])

platformMissing = collect(train[:Platform])
yearMissing = collect(train[:Year_of_Release])
ratingMissing = collect(train[:Rating])
genreMissing = collect(train[:Genre])
publisherMissing = collect(train[:Publisher])
developerMissing = collect(train[:Developer])

14094-element Array{Union{Missing, String},1}:
 "Buddiez- Inc."       
 missing               
 "System 3"            
 "Big Sky Software"    
 "Koei Tecmo Games"    
 "Sega"                
 missing               
 "Beenox"              
 missing               
 missing               
 "EA DICE"             
 "Game Machine Studios"
 missing               
 ⋮                     
 missing               
 "High Impact Games"   
 missing               
 missing               
 missing               
 missing               
 missing               
 "Krome Studios"       
 "Opera House"         
 "Big Sky Software"    
 missing               
 "Full Fat"            

Separation des données par rapport au publisher et Developer

In [89]:
## Remplacer les missings par des string
for i in eachindex(publisherMissing)
    if ismissing(publisherMissing[i])
        publisherMissing[i] = "missing"
    end
end

numGamesByPublisher = countmap(publisherMissing)

publisherNames = collect(keys(numGamesByPublisher))
##va contenir le nombre de jeux associé au publisher du jeux
publisherByCount = []

for i in eachindex(publisherNames)
    for j in eachindex(publisherMissing)
        if publisherMissing[j] == publisherNames[i]
            push!(publisherByCount,numGamesByPublisher[publisherNames[i]])
        end
    end
end
publisherCat = collect(publisherMissing)
for i in eachindex(publisherByCount)
    if publisherByCount[i] == 1
        publisherCat[i] = "one"
    elseif publisherByCount[i] > 1 && publisherByCount[i]<=10
         publisherCat[i] = "between1And10"
    elseif publisherByCount[i] > 10 && publisherByCount[i]<=25
         publisherCat[i] = "between10And25"
    elseif publisherByCount[i] > 25 && publisherByCount[i]<=50
         publisherCat[i] = "between25And50"
    elseif publisherByCount[i] > 50 && publisherByCount[i]<=75
         publisherCat[i] = "between50And75"
    elseif publisherByCount[i] > 75 && publisherByCount[i]<=125
         publisherCat[i] = "between75And125"
    elseif publisherByCount[i] > 125 && publisherByCount[i]<=175
         publisherCat[i] = "between125And175"
    elseif publisherByCount[i] > 175 && publisherByCount[i]<=225
         publisherCat[i] = "between175And225"
    elseif publisherByCount[i] > 225 && publisherByCount[i]<=300
         publisherCat[i] = "between225And300"
    elseif publisherByCount[i] > 300 && publisherByCount[i]<=400
         publisherCat[i] = "between300And400"
    elseif publisherByCount[i] > 400 && publisherByCount[i]<=600
         publisherCat[i] = "between400And600"
    elseif publisherByCount[i] > 600 
        publisherCat[i] = "moreThan600"
    end
end

categoriesPublisher = categorical(publisherCat)
levels(categoriesPublisher)

12-element Array{String,1}:
 "between10And25"  
 "between125And175"
 "between175And225"
 "between1And10"   
 "between225And300"
 "between25And50"  
 "between300And400"
 "between400And600"
 "between50And75"  
 "between75And125" 
 "moreThan600"     
 "one"             

Separation des données par rapport au Developer

In [90]:
## Remplacer les missings par des string
for i in eachindex(developerMissing)
    if ismissing(developerMissing[i])
        developerMissing[i] = "missing"
    end
end

numGamesByDeveloper = countmap(developerMissing)

developerNames = collect(keys(numGamesByDeveloper))
##va contenir le nombre de jeux associé au publisher du jeux
developerByCount = []

for i in eachindex(developerNames)
    for j in eachindex(developerMissing)
        if developerMissing[j] == developerNames[i]
            push!(developerByCount,numGamesByDeveloper[developerNames[i]])
        end
    end
end
developerCat = collect(developerMissing)
for i in eachindex(developerByCount)
    if developerByCount[i] == 1
        developerCat[i] = "one"
    elseif developerByCount[i] > 1 && developerByCount[i]<=10
         developerCat[i] = "between1And10"
    elseif developerByCount[i] > 10 && developerByCount[i]<=25
         developerCat[i] = "between10And25"
    elseif developerByCount[i] > 25 && developerByCount[i]<=50
         developerCat[i] = "between25And50"
    elseif developerByCount[i] > 50 && developerByCount[i]<=75
         developerCat[i] = "between50And75"
    elseif developerByCount[i] > 75 && developerByCount[i]<=125
         developerCat[i] = "between75And125"
    elseif developerByCount[i] > 125 && developerByCount[i]<=175
         developerCat[i] = "between125And175"
    elseif developerByCount[i] > 175 && developerByCount[i]<=225
         developerCat[i] = "between175And225"
    elseif developerByCount[i] > 225 && developerByCount[i]<=300
         developerCat[i] = "between225And300"
    elseif developerByCount[i] > 300 && developerByCount[i]<=400
         developerCat[i] = "between300And400"
    elseif developerByCount[i] > 400 && developerByCount[i]<=600
         developerCat[i] = "between400And600"
    elseif developerByCount[i] > 600 
        developerCat[i] = "moreThan600"
    end
end

categoriesDeveloper = categorical(developerCat)
levels(categoriesDeveloper)

8-element Array{String,1}:
 "between10And25"  
 "between125And175"
 "between1And10"   
 "between25And50"  
 "between50And75"  
 "between75And125" 
 "moreThan600"     
 "one"             

# Analyse exploratoire

Platform, year, rating, genre, critic, user

In [91]:
for i in eachindex(platformMissing) 
    if ismissing(platformMissing[i])
        platformMissing[i] = "NoPlatform"
    end
end

platform = categorical(platformMissing)

for i in eachindex(yearMissing) 
    if ismissing(yearMissing[i])
        yearMissing[i] = 1000
    end
end
year = categorical(yearMissing)

for i in eachindex(ratingMissing) 
    if ismissing(ratingMissing[i])
        ratingMissing[i] = "missingRating"
    end
end
rating = categorical(ratingMissing)

## GENRE

for i in eachindex(genreMissing) 
    if ismissing(genreMissing[i])
        genreMissing[i] = "missing"
    end
end

genre = categorical(genreMissing)

moyenneCriticScore = floor(mean(collect(skipmissing(train[:,:Critic_Score]))))
moyenneCriticCount =floor(mean(collect(skipmissing(train[:,:Critic_Count]))))
moyenneUser_Score = floor(mean(collect(skipmissing(train[:,:User_Score]))))
moyenneUser_Count = floor(mean(collect(skipmissing(train[:,:User_Count]))))


for i in eachindex(Critic_Score) 
    if ismissing(Critic_Score[i]) || ismissing(Critic_Count[i]) || ismissing(User_Score[i]) || ismissing(User_Count[i])
        Critic_Score[i]= moyenneCriticScore
        Critic_Count[i]= moyenneCriticCount
        User_Score[i]= moyenneUser_Score
        User_Count[i]= moyenneUser_Count
    end
end


In [92]:
df1 = DataFrame(Global = Global_Sales,JP = JP_Sales, Platform = platform, Year = year, Rating = rating,
    Other = Other_Sales, CS=Critic_Score, CC=Critic_Count, US=User_Score,UC=User_Count, Genre = genre,Publisher = categoriesPublisher, Developer = categoriesDeveloper)

df2 = DataFrame(NA = NA_Sales,JP = JP_Sales, Platform = platform, Year = year, Rating = rating,
    Other = Other_Sales, CS=Critic_Score, CC=Critic_Count, US=User_Score,UC=User_Count, Genre = genre, Publisher = categoriesPublisher, Developer = categoriesDeveloper)


M₁=lm(@formula(Global ~  JP + Platform + Rating + Year + Other + CS+ CC + US + UC + Genre + Publisher + Developer), df1)
M₂=lm(@formula(NA ~ JP + Platform + Rating + Year + Other +CS+ CC + US + UC + Genre + Publisher + Developer), df2)

println(deviance(M₁))
println(deviance(M₂))
println(adjr2(M₁))
println(adjr2(M₂))

fieldnames(ModelFrame)



8817.531135462621
4694.567568680067
0.7346695408055279
0.5155226961994994


,Global,JP,Platform,Rating,Year,Other,CS,CC,US,UC,Genre,Publisher,Developer
,Float64⍰,Float64⍰,Categorical…⍰,Categorical…⍰,Categorical…⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,Categorical…⍰,Categorical…⍰,Categorical…⍰
1,0.172605,0.0505242,DS,E,2004,0.0100241,74,40,7.5,13,Puzzle,one,between1And10
2,0.0432509,0.0392108,DS,missingRating,2010,0.00179053,69,26,7.0,157,Action,one,between1And10
3,0.0412706,0.000374026,DS,E,2006,0.00179738,69,26,7.0,157,Sports,one,between1And10
4,1.08082,0.00138205,PS2,E,2002,0.14001,69,26,7.0,157,Platform,one,between1And10
5,0.131074,0.0694222,PSV,M,2014,0.00953506,67,19,7.5,61,Action,between1And10,between1And10
6,0.331157,0.14039,PS3,M,2012,0.0302705,72,33,8.2,179,Action,between1And10,between1And10
7,0.164147,0.161401,SNES,missingRating,1997,0.000962246,69,26,7.0,157,Sports,between1And10,between1And10
8,0.168837,0.00168636,X360,E10+,2009,0.011419,63,31,6.6,7,Action,between1And10,between1And10
9,0.0328304,0.0314134,DS,missingRating,2008,0.000182071,69,26,7.0,157,Action,between1And10,between1And10


Extraire la matrice de varible explicative

In [93]:
CS = collect(M₁.mf.df[:,:CS])
CC = collect(M₁.mf.df[:,:CC])
US= collect(M₁.mf.df[:,:US])
UC = collect(M₁.mf.df[:,:UC])
Other = collect(M₁.mf.df[:,:Other])
JP = collect(M₁.mf.df[:,:JP])

Platform = collect(M₁.mf.df[:Platform])
Platform = CategoricalArray(Platform)

Year = collect(M₁.mf.df[:Year])
Year = CategoricalArray(Year)

Rating = collect(M₁.mf.df[:Rating])
Rating = CategoricalArray(Rating)

G = collect(M₁.mf.df[:Genre])
G = CategoricalArray(G)

Pub = collect(M₁.mf.df[:Publisher])
Pub = CategoricalArray(Pub)

Dev= collect(M₁.mf.df[:Developer])
Dev = CategoricalArray(Dev)

fieldnames(CategoricalArray)
k = length(NA_Sales)
X = hcat(CS, US, UC, Other, JP,
    Platform.refs, Year.refs, Rating.refs, G.refs, Pub.refs,Dev.refs)


14094×11 Array{Union{Missing, Float64},2}:
 74.0  7.5    13.0  0.0100241    0.0505242     1.0   1.0  1.0  1.0  1.0  1.0
 69.0  7.0   157.0  0.00179053   0.0392108     1.0   2.0  2.0  2.0  1.0  1.0
 69.0  7.0   157.0  0.00179738   0.000374026   1.0   3.0  1.0  3.0  1.0  1.0
 69.0  7.0   157.0  0.14001      0.00138205    2.0   4.0  1.0  4.0  1.0  1.0
 67.0  7.5    61.0  0.00953506   0.0694222     3.0   5.0  3.0  2.0  2.0  1.0
 72.0  8.2   179.0  0.0302705    0.14039       4.0   6.0  3.0  2.0  2.0  1.0
 69.0  7.0   157.0  0.000962246  0.161401      5.0   7.0  2.0  3.0  2.0  1.0
 63.0  6.6     7.0  0.011419     0.00168636    6.0   8.0  4.0  2.0  2.0  1.0
 69.0  7.0   157.0  0.000182071  0.0314134     1.0   9.0  2.0  2.0  2.0  1.0
 69.0  7.0   157.0  0.0517094    0.000293506   2.0   3.0  2.0  5.0  2.0  1.0
 84.0  7.4  2142.0  0.690459     0.0615083     6.0  10.0  3.0  6.0  2.0  1.0
 69.0  7.0   157.0  0.00124541   0.00158265    7.0  11.0  1.0  2.0  2.0  1.0
 69.0  7.0   157.0  0.0412362    

VIF

In [94]:
#  #Calculer le VIF
# VIF = Float64[] 

# for i=1:p

#     ind = trues(p+1)
#     ind[i+1] = false
#     XX = X[:,ind]
#     YY = X[:,.!(ind)]

#     β̂ = (XX'XX)\XX'YY
#     e = YY-XX*β̂
#     SSE = sum(e.^2)
#     SST = sum((YY-mean(YY)).^2)

#     R² = 1-SSE/SST

#     push!(VIF,1/(1-R²))
    
# end

# VIF

PHI On observe qu'il y a beaucoup de multicolinéarité pcq le 

In [95]:
F = eigfact(X'X)
λ = F[:values]
ϕ = sqrt.(maximum(λ)./λ)

11-element Array{Float64,1}:
 2303.671164404494   
 1340.0908912386199  
  497.6361195701692  
  274.720597507439   
  151.9173925960815  
  120.23703279319723 
  110.25511386270588 
   72.84734696352272 
   55.2395977661488  
    6.084367790923296
    1.0              

Analyse en composante principale

In [96]:
Σ = cov(X)
F = eigen(Σ, scale=false)
λ = F.values
## Les vecteurs propres sont les colonne de la matrice suivante
e = F.vectors
perm = sortperm(λ,rev=true)
λ = λ[perm]
e = e[:,perm]

11×11 Array{Float64,2}:
  0.00685292    0.994565     0.0691023   …  -0.00227636    0.00278796 
  9.35114e-5    0.0641283    0.00704311     -0.0137847    -0.0094827  
  0.999975     -0.00696268   0.0005055      -4.9637e-5     9.12972e-5 
  0.000117183   0.00268032   0.00180582      0.265396     -0.964103   
  3.68262e-5    0.00282118  -0.00531872      0.963961      0.265213   
  0.00195435    0.0687585   -0.465162    …   0.000642021  -0.0028786  
  8.38472e-5    0.0422821   -0.88233        -0.00587122   -0.00189206 
  7.81276e-5   -0.00171644   0.0082426       0.0086752     0.0064076  
  0.000250087   0.010551     0.00956786     -0.00620689   -0.00406065 
 -1.53931e-5   -0.00544766  -0.00912777     -0.000556923  -0.00051825 
 -3.94898e-5    0.0024047   -0.00445848  …  -0.000736772   0.000326044

In [97]:
df1 = DataFrame(k=collect(1:p),λ=λ)
# fig1 = plot(df1,x=:k,y=:λ,Geom.point, Geom.line)

df2 = DataFrame(k=collect(2:p),λ=λ[2:end])
# fig2 = plot(df2,x=:k,y=:λ,Geom.point, Geom.line)

df3 = DataFrame(k=collect(3:p),λ=λ[3:end])
# fig3 = plot(df3,x=:k,y=:λ,Geom.point, Geom.line)

# hstack(fig1,fig2,fig3)

,k,λ
,Int64,Float64
1,3,50.7368
2,4,29.3785
3,5,12.2375
4,6,10.1699
5,7,6.75651
6,8,1.96034
7,9,0.556935
8,10,0.0861197
9,11,0.0293427


In [98]:
# % de variance expliqué en prenant en compte toutes les composantes principales 
p = size(X,2)

ExplainedVars = Float64[]
Σλᵢ = 0
total = sum(λ)
for i in eachindex(λ)
    Σλᵢ= Σλᵢ + λ[i]
    
    push!(ExplainedVars,Σλᵢ/total)
end

df4 = DataFrame(k=collect(1:p),λ=ExplainedVars)
# fig4 = plot(df4,x=:k,y=:λ,Geom.point, Geom.line)
#On voit que la première composante explique pratiquement toute la variabilité,
#ce qui est louche

,k,λ
,Int64,Float64
1,1,0.998534
2,2,0.999132
3,3,0.999526
4,4,0.999753
5,5,0.999848
6,6,0.999927
7,7,0.99998
8,8,0.999995
9,9,0.999999


In [99]:
# % de variance expliqué en prenant en compte toutes les composantes principales sans la
# sans la première qui était trop grande

ExplainedVars = Float64[]
Σλᵢ = 0
λ₂ = collect(λ[2:end])
total = sum(λ₂)
for i in eachindex(λ₂)
    Σλᵢ= Σλᵢ + λ₂[i]
    push!(ExplainedVars,Σλᵢ/total)
end
print(ExplainedVars)
df4 = DataFrame(k=collect(1:10),λ=ExplainedVars)
# fig4 = plot(df4,x=:k,y=:λ,Geom.point, Geom.line)

#On voit qu'on pourrait prendre les composantes 2 à 6, qui explquerait plus de 90% de la variabilité

[0.408051, 0.676419, 0.831814, 0.896544, 0.950336, 0.986074, 0.996443, 0.999389, 0.999845, 1.0]

,k,λ
,Int64,Float64
1,1,0.408051
2,2,0.676419
3,3,0.831814
4,4,0.896544
5,5,0.950336
6,6,0.986074
7,7,0.996443
8,8,0.999389
9,9,0.999845


Prenons les composantes 2 à 5 Pour NA_Sales

In [100]:
# Contruction de la matrice des variables explicatives dans l'espace des composantes principales.
Z = hcat(ones(k),X*e[:,1:3])

# Coefficient de régression dans l'espace des composantes principales
 η̂ = (Z'Z)\Z'NA_Sales

β̂₁₀ = η̂[1]
β̂₁₁ = e[:,1:3]*η̂[2:4]
β̂₁ = vcat(β̂₁₀,β̂₁₁)
# # Affichage de la régression linéaire dans l'espace des composantes principales
# echantillon = layer(x=Z[:,2], y=NA_Sales , Geom.point, Theme(default_color="deepskyblue"))
#  xx = [ones(10) collect(range(110, length=10, stop=135))]
# regression = layer(x=xx[:,2], y = xx*η̂, Geom.line, Theme(default_color="red"))

# plot(echantillon,regression ,
#     Guide.manual_color_key("Légende", ["Échantillon", "Régression"], ["deepskyblue","red"]),
#     Guide.xlabel("Composante principale"), Guide.ylabel("NA_Sales"))


12-element Array{Float64,1}:
 -0.7835343379457749    
  0.013158736549499395  
  0.0008358167388222433 
  0.00030922589041793144
  2.7677959358958207e-5 
  6.39758861060819e-5   
  0.0031806910426248674 
  0.004836241446917316  
 -6.307091935982766e-5  
  9.698544484174529e-5  
 -2.9797663353602123e-5 
  5.4139936541525515e-5 

Prenons les composantes 2 à 5 Pour Global_Sales

In [101]:
# Contruction de la matrice des variables explicatives dans l'espace des composantes principales.
Z = hcat(ones(k),X*e[:,1:3])

# Coefficient de régression dans l'espace des composantes principales
 η̂ = (Z'Z)\Z'Global_Sales 

β̂₂₀ = η̂[1]
β̂₂₁ = e[:,1:3]*η̂[2:4]
β̂₂ = vcat(β̂₂₀,β̂₂₁)
# # Affichage de la régression linéaire dans l'espace des composantes principales
# echantillon = layer(x=Z[:,2], y=NA_Sales , Geom.point, Theme(default_color="deepskyblue"))
#  xx = [ones(10) collect(range(110, length=10, stop=135))]
# regression = layer(x=xx[:,2], y = xx*η̂, Geom.line, Theme(default_color="red"))

# plot(echantillon,regression ,
#     Guide.manual_color_key("Légende", ["Échantillon", "Régression"], ["deepskyblue","red"]),
#     Guide.xlabel("Composante principale"), Guide.ylabel("NA_Sales"))


12-element Array{Float64,1}:
 -1.5204118496514014    
  0.026018483424873708  
  0.0016589873135144462 
  0.0007066243985986359 
  5.872798704740096e-5  
  0.00011290753719801861
  0.0051309877266065874 
  0.007380653033064846  
 -0.00010409089914351877
  0.00021355877647452452
 -8.048058177309847e-5  
  9.56438794202192e-5   

# Charger les données de l'échantillon de test

In [102]:
test = CSV.read("test.csv")
head(test)

,ID,Platform,Year_of_Release,Genre,Publisher,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
,Int64⍰,String⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,String⍰,String⍰
1,14095,GBA,2002,Fighting,Acclaim Entertainment,0.000520983,0.00025736,24,11,3.7,10,Powerhead Games,E
2,14096,PS3,2008,Action,Codemasters,0.000966418,0.0204632,72,35,7.7,27,4J Studios,T
3,14097,N64,1997,Sports,Konami Digital Entertainment,0.259834,0.0489383,missing,missing,missing,missing,missing,missing
4,14098,GC,2002,Fighting,Midway Games,0.00088793,0.00826891,81,20,8.7,43,Midway,M
5,14099,X360,2010,Role-Playing,Sega,0.000439246,0.0388025,63,68,7.2,201,Obsidian Entertainment,M
6,14100,DS,2009,Adventure,Ubisoft,0.001625,0.00016853,missing,missing,7.5,4,Longtail Studios,T


In [103]:
n = size(test,1)

Critic_ScoreTest = collect(test[:,:Critic_Score])
Critic_CountTest = collect(test[:,:Critic_Count])
User_ScoreTest = collect(test[:,:User_Score])
User_CountTest = collect(test[:,:User_Count])
Other_SalesTest = collect(test[:,:Other_Sales])
JP_SalesTest = collect(test[:,:JP_Sales])
genreNonCategorical = collect(test[:,:Genre])
ratingNonCategorical = collect(test[:Rating])
platformNonCategorical = collect(test[:Platform])
yearNonCategorical = collect(test[:Year_of_Release])
publisherTest = collect(test[:Publisher])
developerTest = collect(test[:Developer])

2490-element Array{Union{Missing, String},1}:
 "Powerhead Games"                  
 "4J Studios"                       
 missing                            
 "Midway"                           
 "Obsidian Entertainment"           
 "Longtail Studios"                 
 "NanaOn-Sha"                       
 "InXile Entertainment"             
 missing                            
 "Rovio Mobile- Rovio Entertainment"
 "Namco"                            
 "Gratuitous Games"                 
 "Ubisoft Paris"                    
 ⋮                                  
 "Sports Interactive"               
 missing                            
 "Sidhe Interactive"                
 "Capcom"                           
 missing                            
 "BudCat"                           
 "EA Canada"                        
 missing                            
 "7 Studios"                        
 missing                            
 missing                            
 missing                     

On filtre les publishers

In [104]:
## Remplacer les missings par des string
for i in eachindex(publisherTest)
    if ismissing(publisherTest[i])
        publisherTest[i] = "missing"
    end
end

numGamesByPublisher = countmap(publisherTest)

publisherNames = collect(keys(numGamesByPublisher))
##va contenir le nombre de jeux associé au publisher du jeux
publisherByCount = []

for i in eachindex(publisherNames)
    for j in eachindex(publisherTest)
        if publisherTest[j] == publisherNames[i]
            push!(publisherByCount,numGamesByPublisher[publisherNames[i]])
        end
    end
end
publisherCat = collect(publisherTest)
for i in eachindex(publisherByCount)
    if publisherByCount[i] == 1
        publisherCat[i] = "one"
    elseif publisherByCount[i] > 1 && publisherByCount[i]<=10
         publisherCat[i] = "between1And10"
    elseif publisherByCount[i] > 10 && publisherByCount[i]<=25
         publisherCat[i] = "between10And25"
    elseif publisherByCount[i] > 25 && publisherByCount[i]<=50
         publisherCat[i] = "between25And50"
    elseif publisherByCount[i] > 50 && publisherByCount[i]<=75
         publisherCat[i] = "between50And75"
    elseif publisherByCount[i] > 75 && publisherByCount[i]<=125
         publisherCat[i] = "between75And125"
    elseif publisherByCount[i] > 125 && publisherByCount[i]<=175
         publisherCat[i] = "between125And175"
    elseif publisherByCount[i] > 175 && publisherByCount[i]<=225
         publisherCat[i] = "between175And225"
    elseif publisherByCount[i] > 225 && publisherByCount[i]<=300
         publisherCat[i] = "between225And300"
    elseif publisherByCount[i] > 300 && publisherByCount[i]<=400
         publisherCat[i] = "between300And400"
    elseif publisherByCount[i] > 400 && publisherByCount[i]<=600
         publisherCat[i] = "between400And600"
    elseif publisherByCount[i] > 600 
        publisherCat[i] = "moreThan600"
    end
end

categoriesPublisherTest = categorical(publisherCat)
levels(categoriesPublisherTest)

8-element Array{String,1}:
 "between10And25"  
 "between125And175"
 "between175And225"
 "between1And10"   
 "between25And50"  
 "between50And75"  
 "between75And125" 
 "one"             

On filtre les developpeur 

In [105]:
## Remplacer les missings par des string
for i in eachindex(developerTest)
    if ismissing(developerTest[i])
        developerTest[i] = "missing"
    end
end

numGamesByDeveloper = countmap(developerTest)

developerNames = collect(keys(numGamesByDeveloper))
##va contenir le nombre de jeux associé au publisher du jeux
developerByCount = []

for i in eachindex(developerNames)
    for j in eachindex(developerTest)
        if developerTest[j] == developerNames[i]
            push!(developerByCount,numGamesByDeveloper[developerNames[i]])
        end
    end
end
developerCat = collect(developerTest)
for i in eachindex(developerByCount)
    if developerByCount[i] == 1
        developerCat[i] = "one"
    elseif developerByCount[i] > 1 && developerByCount[i]<=10
         developerCat[i] = "between1And10"
    elseif developerByCount[i] > 10 && developerByCount[i]<=25
         developerCat[i] = "between10And25"
    elseif developerByCount[i] > 25 && developerByCount[i]<=50
         developerCat[i] = "between25And50"
    elseif developerByCount[i] > 50 && developerByCount[i]<=75
         developerCat[i] = "between50And75"
    elseif developerByCount[i] > 75 && developerByCount[i]<=125
         developerCat[i] = "between75And125"
    elseif developerByCount[i] > 125 && developerByCount[i]<=175
         developerCat[i] = "between125And175"
    elseif developerByCount[i] > 175 && developerByCount[i]<=225
         developerCat[i] = "between175And225"
    elseif developerByCount[i] > 225 && developerByCount[i]<=300
         developerCat[i] = "between225And300"
    elseif developerByCount[i] > 300 && developerByCount[i]<=400
         developerCat[i] = "between300And400"
    elseif developerByCount[i] > 400 && developerByCount[i]<=600
         developerCat[i] = "between400And600"
    elseif developerByCount[i] > 600 
        developerCat[i] = "moreThan600"
    end
end

categoriesDeveloperTest = categorical(developerCat)
levels(categoriesDeveloperTest)

5-element Array{String,1}:
 "between10And25"
 "between1And10" 
 "between25And50"
 "moreThan600"   
 "one"           

On filtre les critic Score, User Score, 
Critic Count, User Count, Genre, Rating, Platform

In [106]:

for i in eachindex(Critic_ScoreTest) 
    if ismissing(Critic_ScoreTest[i]) || ismissing(Critic_CountTest[i]) || ismissing(User_ScoreTest[i]) || ismissing(User_CountTest[i])
        Critic_ScoreTest[i]= moyenneCriticScore
        Critic_CountTest[i]= moyenneCriticCount
        User_ScoreTest[i]= moyenneUser_Score
        User_CountTest[i]= moyenneUser_Count
    end
end

## Platforme

for i in eachindex(platformNonCategorical)
    if(platformNonCategorical[i] == "GG")
        platformNonCategorical[i] = "PC"
    end
end

platformTest = categorical(platformNonCategorical)

## Year

# On eleve les donnes manquantes
for i in eachindex(yearNonCategorical)
    if(ismissing(yearNonCategorical[i]))
        yearNonCategorical[i] = 1000
    else
        if yearNonCategorical[i] == 2020
            yearNonCategorical[i] = 1996
        elseif yearNonCategorical[i] == 1994
            yearNonCategorical[i] = 1996
        end
    end
end


yearTest = categorical(yearNonCategorical)


for i in eachindex(ratingNonCategorical)
    if(ismissing(ratingNonCategorical[i]))
        ratingNonCategorical[i] = "missingRating"
    end
end

ratingTest = categorical(ratingNonCategorical)

##Genre


for i in eachindex(genreNonCategorical)
    if(ismissing(genreNonCategorical[i]))
        genreNonCategorical[i] = "missing"
    end
end

genreTest = categorical(genreNonCategorical)

2490-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "Fighting"    
 "Action"      
 "Sports"      
 "Fighting"    
 "Role-Playing"
 "Adventure"   
 "Simulation"  
 "Role-Playing"
 "Fighting"    
 "Puzzle"      
 "Fighting"    
 "Sports"      
 "Shooter"     
 ⋮             
 "Sports"      
 "Action"      
 "Racing"      
 "Action"      
 "Shooter"     
 "Sports"      
 "Sports"      
 "Adventure"   
 "Adventure"   
 "Strategy"    
 "Fighting"    
 "Sports"      

Prediction

In [107]:
X = hcat(Critic_ScoreTest, User_ScoreTest, User_CountTest, Other_SalesTest, JP_SalesTest,
    platformTest.refs, yearTest.refs, ratingTest.refs, genreTest.refs, categoriesPublisherTest.refs,categoriesDeveloperTest.refs)
Xtest = hcat(ones(n),X)

NA_SalesTest = Xtest*β̂₁
Global_SalesTest = Xtest*β̂₂

Predict()


2490-element Array{Union{Missing, Float64},1}:
 -0.8701274045366565 
  0.41003070946726433
  0.43532714886333296
  0.6596208948181317 
  0.3283768961463075 
  0.4659957720158197 
  0.10817728488740673
  0.5694931515366504 
  0.49231493593082387
  0.5063950020032773 
  1.1042622080411206 
  0.6048878445939688 
  0.5015778277179939 
  ⋮                  
  1.026947001251439  
  0.5341031362496127 
 -0.2408998750712288 
  1.239841057006897  
  0.5052320850700593 
  0.41067193751783043
  0.6603929451406847 
  0.5530184036903946 
 -0.07581064016641914
  0.42757325096556537
  0.48933384389630796
  0.4432483478044635 

In [109]:
dfTest = DataFrame(one = ones(n),JP = JP_SalesTest, Platform = platformTest, Rating = ratingTest, Year = yearTest, Other = Other_SalesTest,CS = Critic_ScoreTest, CC=Critic_CountTest, US=User_ScoreTest,
    UC=User_CountTest, Genre = genreTest, Publisher = categoriesPublisherTest, Developer = categoriesDeveloperTest )

NA_SalesTest = predict(M₂,dfTest)

Global_SalesTest = predict(M₁,dfTest)

2490-element Array{Union{Missing, Float64},1}:
  0.05983970830038835 
  0.23721902651606439 
  1.2065014381919847  
  0.3290623486523103  
  0.4624957301240658  
  0.05020832182864743 
  2.7572567007493363  
  0.04816155952162682 
 -0.034914219400833946
  0.32341766959382395 
  1.1071482957285534  
  0.3219734725524959  
  0.3969995545813584  
  ⋮                   
  1.1246130354100914  
  0.268757352783601   
 -0.12104528242424256 
  6.453352003749379   
  0.12883640560802764 
  0.36689356868808715 
  0.30492714307904534 
  0.2708232636687262  
  0.017293536692366697
  0.3141151945131846  
  0.2640566564266863  
  0.38565935529544004 

In [110]:


predictions = DataFrame(ID = IDtest, NA_Sales = NA_SalesTest, Global_Sales = Global_SalesTest)
CSV.write("prediction_ComposantePrincipale.csv",predictions)


"prediction_ComposantePrincipale.csv"